# API documentation 
https://cloud.ibm.com/apidocs/assistant-v2?code=python#listassistants


In [9]:
import pprint
from dotenv import load_dotenv
import os
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import pandas as pd
import pickle

load_dotenv()

True

# API assistant setup

In [10]:
API_KEY = os.getenv(key='API_KEY')
URL = 'https://api.us-east.assistant.watson.cloud.ibm.com/'
ASSISTANT_NAME = 'Ask_Proc_Main'

In [11]:

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(URL)

# Get List of assistants available

This will give you all the assistants that are available in that instance in IBM cloud. 

In [12]:
assistants_list = assistant.list_assistants()

In [13]:
for bot in assistants_list.get_result().get('assistants'):
    if bot.get('name') == ASSISTANT_NAME:
        ASSISTANT_ID = bot.get('assistant_id')
        pprint.pprint(bot)

print(f'\n\nYour assistant id is: {ASSISTANT_ID}')

{'assistant_environments': [{'environment': 'draft',
                             'environment_id': '572332fa-74b2-4e42-9c4d-93c891106824',
                             'name': 'draft'},
                            {'environment': 'live',
                             'environment_id': '646fce51-a4ec-4bc8-8928-ef4eb222a59a',
                             'name': 'live'}],
 'assistant_id': '9dcbdb4b-f743-4a96-b20b-51f80f9ec4d7',
 'assistant_skills': [{'skill_id': '79fa487e-fc14-4f85-89b9-8f00c88f0ff7',
                       'type': 'dialog'},
                      {'skill_id': '5cbe6f41-8364-4f69-9e0a-1e7891cdde76',
                       'type': 'action'}],
 'description': 'Main Assistant for Ask Procurement',
 'language': 'en',
 'multi_lingual': {'enabled': False},
 'name': 'Ask_Proc_Main'}


Your assistant id is: 9dcbdb4b-f743-4a96-b20b-51f80f9ec4d7


In [14]:
# here you can define the assistant environment to get the logs from 
ASSISTANT_ENV_ID = '572332fa-74b2-4e42-9c4d-93c891106824'

# Get log data 

In [15]:
# to get all the logs 
cursor = None
page = 1

all_logs = []

while True:
    print(f'working on page {page}')
    page += 1
    response=assistant.list_logs(
        assistant_id=ASSISTANT_ENV_ID,
        cursor=cursor,
    ).get_result()

    all_logs.extend(response['logs'])

    cursor=response.get('pagination').get('next_cursor')
    if not cursor:
        break

working on page 1
working on page 2
working on page 3
working on page 4
working on page 5
working on page 6
working on page 7
working on page 8
working on page 9
working on page 10
working on page 11
working on page 12
working on page 13
working on page 14
working on page 15
working on page 16
working on page 17
working on page 18
working on page 19
working on page 20
working on page 21
working on page 22
working on page 23
working on page 24
working on page 25
working on page 26
working on page 27
working on page 28
working on page 29
working on page 30
working on page 31
working on page 32
working on page 33
working on page 34
working on page 35
working on page 36
working on page 37
working on page 38
working on page 39
working on page 40
working on page 41
working on page 42
working on page 43
working on page 44
working on page 45
working on page 46
working on page 47
working on page 48
working on page 49
working on page 50
working on page 51
working on page 52
working on page 53
wo

In [7]:
#TEST
response=assistant.list_logs(
        assistant_id=ASSISTANT_ENV_ID,
    ).get_result()

In [16]:
len(all_logs)

9457

#to sabe the logs in picke object 
with open('all_logs.pkl', 'wb') as file: 
    pickle.dump(all_logs, file=file)

# From here to read all the logs from pkl file 

In [53]:
#read pickle back 
with open('all_logs.pkl', 'rb') as file:
    # Load the pickle object using pickle.load
    all_logs = pickle.load(file)

In [19]:
len(all_logs)

9457

In [54]:
type(all_logs)

list

In [55]:
all_logs[0]

{'log_id': '6521d40b-2e63-4b68-919c-4537cf7d9c4d',
 'request': {'input': {'options': {'alternate_responses': True,
    'debug': True,
    'alternate_intents': True,
    'disambiguation': {'alternate_responses': False},
    'async_callout': True,
    'suggestion_only': False,
    'return_context': True,
    'export': True,
    'skip_user_input': True},
   'message_type': 'text',
   'source': {'id': 'anonymous_IBMuid-2bb715f0-9a44-4abc-a324-9d9e49377a77',
    'type': 'user'},
   'text': ''},
  'context': {'skills': {'main skill': {'system': {}},
    'actions skill': {'system': {}, 'user_defined': {}}},
   'metadata': {'user_id': 'anonymous_IBMuid-2bb715f0-9a44-4abc-a324-9d9e49377a77'},
   'system': {'dialog_request_counter': 1,
    'dialog_turn_counter': 1,
    'assistant_id': '572332fa-74b2-4e42-9c4d-93c891106824'},
   'conversation_id': '7578e4f0-6a5d-4597-9018-1e674f911e38',
   'global': {'system': {'user_id': 'anonymous_IBMuid-2bb715f0-9a44-4abc-a324-9d9e49377a77',
     'timezone': '

In [52]:
log = 9435
print('REQUEST')
pprint.pprint(all_logs[log].get('request').get('input'))
print('RESPONSE:')
pprint.pprint(all_logs[log].get('response').get('output'))


REQUEST
{'internal': {'callout_count': {'step_806_result_1': 0},
              'callout_duration': 2245,
              'callout_request': {'body': {'invoice_number': '000000000000393-EX,0005369289,001200115526-01,0094874223,0094960918,',
                                           'user_id': 'sergioev@mx1.ibm.com'},
                                  'headers': {'Content-Type': 'application/json',
                                              'accept': 'application/json'},
                                  'method': 'POST',
                                  'path': '/invoice_summary',
                                  'query_parameters': '',
                                  'url': 'https://epm-token-api-test-emailsenderfunctionallity.5a8cbfd1.public.multi-containers.ibm.com'},
              'callout_result_variable': 'step_806_result_1',
              'integration_callback': {'ip_address': '172.30.79.160',
                                       'request_id': '33ef6fcf-1e1a-45aa-9507-6f5

In [ ]:
# Get all keys for requests 
request_first_level_keys = set()

for log in all_logs:
    data = log.get('request').get('input')
    keys = data.keys()
    for k in keys:
        request_first_level_keys.add(k)

In [ ]:
request_first_level_keys

In [ ]:
# count all keys from request 

keys_counter = {
    'entities': 0,
    'filter': 0,
    'intents': 0,
    'internal': 0,
    'message_type': 0,
    'options': 0,
    'query_type': 0,
    'return_to_dialog': 0,
    'source': 0,
    'suggestion_id': 0,
    'text': 0
}

for log in all_logs:
    data = log.get('request').get('input')
    keys = data.keys()
    for k in keys_counter.keys():
        if k in keys:
            keys_counter[k] += 1



In [ ]:
keys_counter

In [ ]:
# anlyze the data in each key for request 
count = 0

for log in all_logs:
    data = log.get('request').get('input')
    keys = data.keys()
    if 'suggestion_id' in keys:
        pprint.pprint(data)
        print('--------END--------')
        count += 1
    if count == 7:
        break



In [ ]:
all_logs[0].get('request').get('input')

In [ ]:
def json_expander(base_dict):
    my_data = {}
    change_counts = 0
    for key, val in base_dict.items():
        if type(val) == dict:
            change_counts += 1
            for key2, val2 in val.items():
                my_data[key+'_'+key2] = val2
        if type(val) == str:
            my_data[key] = val
        if type(val) == list:
            if (len(val)>0) and (type(val[0]) == dict):
                change_counts += 1
                for key3, val3 in val[0].items():
                    my_data[key+'_'+key3] = val3
            if (len(val)>0) and (type(val[0]) == str):
                my_data[key] = val[0]

    return my_data, change_counts

In [ ]:
all_logs[0].get('response').get('output')

In [ ]:
df = pd.DataFrame()
data = {
    'log_id': all_logs[0].get('log_id'),
    'input': all_logs[0].get('request').get('input')
    }

data


In [ ]:
counter_while = 0
change_counter = 1 

while change_counter != 0:
    if counter_while == 0:
        new_data, change_counter = json_expander(base_dict=data)
        counter_while += 1
    else: 
        new_data, change_counter = json_expander(base_dict=new_data)
    


In [ ]:
new_data

# How to save logs in able or csv 

In [ ]:
df_logs = pd.DataFrame()

for log in all_logs: 
    temp = {
        'log_id' : log.get('log_id'),
        'request_timestamp' : log.get('request_timestamp'),
        'response_timestamp' : log.get('response_timestamp'),
        'assistant_id' : log.get('assistant_id'),
        'session_id' : log.get('session_id'),
        'customer_id' : log.get('customer_id'),
        'skill_id' : log.get('skill_id'),
        'request_input' : log.get('request').get('input'),
        'response_output' : log.get('response').get('output')
    }
    counter_while = 0
    change_counter = 1 

    while change_counter != 0:
        if counter_while == 0:
            new_data, change_counter = json_expander(base_dict=temp)
            counter_while += 1
        else: 
            new_data, change_counter = json_expander(base_dict=new_data)

    temp_df = pd.DataFrame([new_data])
    #print(temp_df['input_source_id'][0])

    df_logs = pd.concat([df_logs, temp_df], ignore_index=True)

    


In [ ]:
all_logs[0].keys()

In [ ]:
for k in all_logs[0].keys():
    print(k)

In [ ]:
all_logs[500].get('request').get('input')

In [ ]:
all_logs[0].get('request').get('input').keys()

In [ ]:
df_logs['request_timestamp'] = pd.to_datetime(df_logs['request_timestamp'], utc=False)#.dt.floor('ms')
df_logs['request_timestamp'] = df_logs['request_timestamp'].dt.tz_localize(None)
df_logs['response_timestamp'] = pd.to_datetime(df_logs['response_timestamp'], utc=False)#.dt.floor('ms')
df_logs['response_timestamp'] = df_logs['response_timestamp'].dt.tz_localize(None)
df_logs.sort_values(inplace=True, by=['session_id', 'request_timestamp'], ascending=[True, True])
df_logs.reset_index(inplace=True, drop=True)
df_logs.head()

In [ ]:
df_logs.columns

In [ ]:
df_logs.to_csv('df_logs.csv', index=False, sep='|')
#df_logs.to_parquet('df_logs.gzip', index=False, compression='gzip')

In [ ]:
!pip install openpyxl

In [ ]:
df_logs.to_excel('df_logs.xlsx', index=False)

In [ ]:
all_logs[9000].keys()

In [ ]:
all_logs[9001].get('customer_id')

In [ ]:
all_logs[9000].get('request').get('input')

In [ ]:
all_logs[9000].get('response').get('output')

In [ ]:
logs_draft = assistant.list_logs(assistant_id=ASSISTANT_ENV_ID)

In [ ]:
# saving the logs data in a variable
logs_results_data = logs_draft.get_result()

In [ ]:
# what keys do we have in the dictionay ? 
logs_results_data.keys()

In [ ]:
logs_results_data['pagination']

In [ ]:
len(logs_results_data['logs'])

In [ ]:
#log results (only one element of the log list)
logs_results_data['logs']